In [22]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0

# Load pre-trained EfficientNetB0 model without the top classification layer
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [23]:
x = base_model.output
model = tf.keras.Model(inputs = base_model.input, outputs = x)
model.summary()

Model: "model_5"


__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_6 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 rescaling_6 (Rescaling)     (None, 224, 224, 3)          0         ['input_6[0][0]']             
                                                                                                  
 normalization_3 (Normaliza  (None, 224, 224, 3)          7         ['rescaling_6[0][0]']         
 tion)                                                                                            
                                                                                                  
 rescaling_7 (Rescaling)     (None, 224, 224, 3)          0         ['normalization_3[0][0]']     
          

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="q4L4ubfqzvMFfhfNMI9j")
project = rf.workspace("roboflow-58fyf").project("rock-paper-scissors-sxsw")
dataset = project.version(12).download("tensorflow")

In [7]:
cd C:\VS_Code_Programs\2022A8PS1238P\Rock-Paper-Scissors-SXSW-12

C:\VS_Code_Programs\2022A8PS1238P\Rock-Paper-Scissors-SXSW-12


In [8]:
import pandas as pd
df = pd.read_csv('train/_annotations.csv')
df.dropna()

,filename,width,height,class,xmin,ymin,xmax,ymax
0,zoom_tests_RockPaperScissors-mohamed_mp4-67_jp...,640,640,Rock,12,0,141,250
1,egohands-public-1625070649053_png_jpg.rf.000a5...,640,640,Rock,153,407,485,637
2,zoom_tests_RockPaperScissors-mohamed_mp4-111_j...,640,640,Paper,0,227,233,640
3,IMG_5636_MOV-61_jpg.rf.003476f680b71b74f3764a1...,640,640,Rock,28,367,175,467
4,IMG_5636_MOV-61_jpg.rf.003476f680b71b74f3764a1...,640,640,Paper,281,333,556,452
...,...,...,...,...,...,...,...,...
7816,IMG_5567_mp4-163_jpg.rf.c7c964e5f3334f60c8f6f6...,640,640,Paper,416,305,623,430
7817,Screen-Shot-2022-02-08-at-1-00-58-PM_png.rf.c7...,640,640,Scissors,461,179,640,477
7818,IMG_5567_mp4-89_jpg.rf.c7b9f669eca26311965b115...,640,640,Paper,220,381,435,519
7819,IMG_7043_MOV-176_jpg.rf.c7cc43070427a0b1b23af9...,640,640,Rock,0,226,520,526


In [9]:
# A Function to get the name of the image

def id_to_path(split, filename):
  path = "C:/VS_Code_Programs/2022A8PS1238P/Rock-Paper-Scissors-SXSW-12/" + split + "/" + filename
  return path

# A function to resize image
import tensorflow as tf

def resize_image(input_path):
    image = tf.io.read_file(input_path)
    image = tf.image.decode_image(image, channels=3)  # Ensure 3 channels (RGB)

    # Resize the image to (224, 224, 3)
    resized_image = tf.image.resize(image, (224, 224))

    return resized_image

In [14]:
data = {
    'image': [],
    'class': [],
    'xmin': [],
    'ymin': [],
    'xmax': [],
    'ymax': []
}

dataframe = pd.DataFrame(data)

for index, row in df.iterrows():
  df_new_row = pd.DataFrame({'image': [(resize_image(id_to_path("train", row['filename'])))], 'class': row['class'], 'xmin': row['xmin'], 'ymin': row['ymin'], 'xmax': row['xmax'], 'ymax': row['ymax']})
  dataframe = pd.concat([dataframe, df_new_row])

dataframe.reset_index(inplace = True)
dataframe = dataframe.drop('index', axis = 1)
print(dataframe)

                                                  image     class   xmin  \
0     (((tf.Tensor(242.30612, shape=(), dtype=float3...      Rock   12.0   
1     (((tf.Tensor(93.07653, shape=(), dtype=float32...      Rock  153.0   
2     (((tf.Tensor(195.29591, shape=(), dtype=float3...     Paper    0.0   
3     (((tf.Tensor(37.0, shape=(), dtype=float32), t...      Rock   28.0   
4     (((tf.Tensor(37.0, shape=(), dtype=float32), t...     Paper  281.0   
...                                                 ...       ...    ...   
7816  (((tf.Tensor(177.07143, shape=(), dtype=float3...     Paper  416.0   
7817  (((tf.Tensor(160.07143, shape=(), dtype=float3...  Scissors  461.0   
7818  (((tf.Tensor(176.9949, shape=(), dtype=float32...     Paper  220.0   
7819  (((tf.Tensor(96.0, shape=(), dtype=float32), t...      Rock    0.0   
7820  (((tf.Tensor(71.0, shape=(), dtype=float32), t...     Paper  306.0   

       ymin   xmax   ymax  
0       0.0  141.0  250.0  
1     407.0  485.0  637.0  
2  

In [15]:
# One-hot encoding

dataframe = pd.get_dummies(dataframe, columns=['class'], prefix=['class'])

print(dataframe)

                                                  image   xmin   ymin   xmax  \
0     (((tf.Tensor(242.30612, shape=(), dtype=float3...   12.0    0.0  141.0   
1     (((tf.Tensor(93.07653, shape=(), dtype=float32...  153.0  407.0  485.0   
2     (((tf.Tensor(195.29591, shape=(), dtype=float3...    0.0  227.0  233.0   
3     (((tf.Tensor(37.0, shape=(), dtype=float32), t...   28.0  367.0  175.0   
4     (((tf.Tensor(37.0, shape=(), dtype=float32), t...  281.0  333.0  556.0   
...                                                 ...    ...    ...    ...   
7816  (((tf.Tensor(177.07143, shape=(), dtype=float3...  416.0  305.0  623.0   
7817  (((tf.Tensor(160.07143, shape=(), dtype=float3...  461.0  179.0  640.0   
7818  (((tf.Tensor(176.9949, shape=(), dtype=float32...  220.0  381.0  435.0   
7819  (((tf.Tensor(96.0, shape=(), dtype=float32), t...    0.0  226.0  520.0   
7820  (((tf.Tensor(71.0, shape=(), dtype=float32), t...  306.0  148.0  630.0   

       ymax  class_Paper  class_Rock  c

In [16]:
import numpy as np

x_train_classification = np.stack((dataframe['image']))
y_train_classification = np.stack(dataframe['class_Rock', 'class_Scissor', 'class_Paper' ])

MemoryError: Unable to allocate 4.39 GiB for an array with shape (7821, 224, 224, 3) and data type float32

In [ ]:
batch_size = 64
epochs = 10

model.fit(x_train_classification, y_train_classification, batch_size = batch_size, epochs = epochs, validation_split = 0.1)

In [ ]:
# Create a Region Proposal Network with input as output of Image classification model

from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Dense

rpn_conv = Conv2D(512, (3, 3), padding='same', activation='relu')(input_feature_maps)
rpn_cls = Conv2D(9, (1, 1), activation='sigmoid')(rpn_conv)
rpn_reg = Conv2D(36, (1, 1))(rpn_conv)

rpn_model = tf.keras.Model(inputs=x, outputs=[rpn_cls, rpn_reg])